# Final Challenge #QuarentenaDados

### Imported libraries

In [0]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import *
from sklearn.neural_network import *
from sklearn.tree import *
from sklearn.svm import *
from sklearn.neighbors import *
from sklearn.gaussian_process import *
from sklearn.kernel_ridge import *
from sklearn.isotonic import *
from sklearn.ensemble import *

### Reading the data

In [0]:
URI_TRAIN = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TEST = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_CHALLENGEQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

train_data = pd.read_csv(URI_TRAIN)
test_data = pd.read_csv(URI_TEST)
challengeqt_data = pd.read_csv(URI_CHALLENGEQT)

train_error = "Error while loading training data"
test_error = "Error while loading testing data"
challengeqt_error = "Error while loading submission data"

assert train_data.shape == (150000, 5), train_error
assert test_data.shape == (20000, 5), test_error
assert challengeqt_data.shape == (10000, 5), challengeqt_error

### Division of data into training and test sets

In [0]:
column_label = 'NU_NOTA_LC'
column_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_train = dados_treino[column_features].to_numpy()
Y_train = dados_treino[column_label].to_numpy()
X_test = dados_teste[column_features].to_numpy()
Y_test = dados_teste[column_label].to_numpy()

### Ensemble training consisting of regression models Random Forest e Multilayer Perceptron (MLP)

In [0]:
m1 = MLPRegressor(max_iter = 500, learning_rate_init = 0.004, hidden_layer_sizes = (20,), random_state = 24, learning_rate = 'invscaling', solver = 'adam', batch_size = 64)
m2 = RandomForestRegressor(n_estimators=89, max_depth=10, max_features='sqrt', random_state=23)

models = [('lr', m1), ('rf', m2)]
ensemble = VotingRegressor(models)
ensemble.fit(X_train, Y_train)
predictions = ensemble.predict(X_test)
mse = mean_squared_error(Y_test, predictions)
print(mse)

2052.4518367697956


### Creation of the output file containing the result

In [0]:
from google.colab import files

MODELO = ensemble
X_desafioqt = dados_desafioqt[coluna_features].to_numpy()
predicao_desafioqt = ensemble.predict(X_desafioqt)

desafio_df = pd.DataFrame(dados_desafioqt.ID)
desafio_df[coluna_label] = predicao_desafioqt

desafio_df.to_csv('PREDICAO_DESAFIOQT.csv', index=False) 
files.download('PREDICAO_DESAFIOQT.csv')